In [1]:
import initialize_notebook
import pandas as pd

from ams.DateRange import DateRange
%matplotlib inline
from pathlib import Path

from pyspark.sql import functions as F
from pyspark.sql import types as T

from ams.config import constants
from ams.services import spark_service
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
from ams.services import file_services
twitter_folder = 'twitter'

learning_prep_dir = Path(constants.DATA_PATH, twitter_folder, "learning_prep_drop", "learning_prep_drop_2020-09-27_23-16-06-769.42")
# learning_prep_dir = Path(constants.DATA_PATH, twitter_folder, "learning_prep_drop", "learning_prep_drop_2020-08-31_03-22-33-670.81")

source_path = file_services.get_single_file_from_path(learning_prep_dir, ".csv")

Setting up logging...
Will use logging path: C:\Users\Chris\workspaces\data\logs\alpha_media_signal


In [3]:
%%time

df_twitter = pd.read_csv(str(source_path))
# df_twitter = df_twitter.iloc[:1000]

print(df_twitter.shape[0])

<string>:2: DtypeWarning: Columns (9,16,23,26,31) have mixed types.Specify dtype option on import or set low_memory=False.


4354341
Wall time: 44.2 s


In [4]:
%%time

from ams.services import ticker_service

ttd = ticker_service.extract_ticker_tweet_dates(df_twitter)
df_stock_data = ticker_service.get_ticker_on_dates(ttd)

print(df_stock_data.shape[0])

No data for ticker 'ABAN' in ['2020-08-26', '2020-08-17', '2020-08-16', '2020-08-15', '2020-08-13', '2020-08-12', '2020-09-06', '2020-09-05', '2020-09-04', '2020-09-07', '2020-09-09', '2020-09-08', '2020-09-11', '2020-09-10', '2020-09-12', '2020-09-13', '2020-09-15', '2020-09-19', '2020-09-20', '2020-09-18', '2020-09-22']
No data for ticker 'ABOV' in ['2020-09-14']
No data for ticker 'ACEC' in ['2020-09-16']
No data for ticker 'ACHI' in ['2020-09-13']
No data for ticker 'ADBL' in ['2020-08-18', '2020-08-21', '2020-08-25', '2020-08-11', '2020-08-17', '2020-08-19', '2020-08-23', '2020-08-20', '2020-08-10', '2020-08-26', '2020-08-13', '2020-08-24', '2020-08-16', '2020-08-14', '2020-08-12', '2020-08-15', '2020-09-09', '2020-09-03', '2020-09-04', '2020-09-10', '2020-09-05', '2020-09-07', '2020-09-02', '2020-09-11', '2020-09-06', '2020-09-14', '2020-09-13', '2020-09-16', '2020-09-18', '2020-09-19', '2020-09-21', '2020-09-20', '2020-09-22']
No data for ticker 'ADEX' in ['2020-08-26', '2020-08

No data for ticker 'ASHW' in ['2020-08-17', '2020-08-11', '2020-08-25', '2020-08-15', '2020-08-21', '2020-08-16', '2020-09-05', '2020-09-04', '2020-09-08', '2020-09-11', '2020-09-10', '2020-09-12', '2020-09-13', '2020-09-15', '2020-09-16', '2020-09-18', '2020-09-17', '2020-09-19', '2020-09-21', '2020-09-22']
No data for ticker 'ASKJ' in ['2020-09-22']
No data for ticker 'ASVI' in ['2020-08-16', '2020-08-15', '2020-08-26', '2020-08-20', '2020-08-12', '2020-08-13', '2020-08-11', '2020-08-17', '2020-08-23', '2020-08-25', '2020-08-24', '2020-08-14', '2020-08-10', '2020-08-21', '2020-08-18', '2020-09-04', '2020-09-03', '2020-09-11', '2020-09-10', '2020-09-06', '2020-09-07', '2020-09-08', '2020-09-09', '2020-09-05', '2020-09-14', '2020-09-13', '2020-09-12', '2020-09-16', '2020-09-17', '2020-09-18', '2020-09-15', '2020-09-19', '2020-09-20', '2020-09-22', '2020-09-21']
No data for ticker 'ATGC' in ['2020-08-25', '2020-08-14', '2020-08-16', '2020-08-10', '2020-09-04', '2020-09-07', '2020-09-10'

No data for ticker 'BRL' in ['2020-08-25', '2020-08-26', '2020-08-24', '2020-09-10', '2020-09-11', '2020-09-09', '2020-09-13']
No data for ticker 'BSBN' in ['2020-08-17', '2020-08-16', '2020-08-26', '2020-09-04', '2020-09-07', '2020-09-06', '2020-09-05', '2020-09-08', '2020-09-10', '2020-09-12', '2020-09-13', '2020-09-14', '2020-09-16', '2020-09-20', '2020-09-22']
No data for ticker 'BTGI' in ['2020-08-23', '2020-08-17', '2020-09-04', '2020-09-06', '2020-09-12', '2020-09-11', '2020-09-16']
No data for ticker 'BTRX' in ['2020-09-10', '2020-09-17', '2020-09-19']
No data for ticker 'BUCA' in ['2020-08-21', '2020-08-20', '2020-08-19', '2020-08-11', '2020-08-14', '2020-08-26', '2020-08-12', '2020-08-16', '2020-08-13', '2020-08-10', '2020-08-23', '2020-08-15', '2020-08-17', '2020-08-22', '2020-08-25', '2020-08-09', '2020-08-24', '2020-08-18', '2020-09-04', '2020-09-03', '2020-09-05', '2020-09-02', '2020-09-09', '2020-09-06', '2020-09-08', '2020-09-10', '2020-09-07', '2020-09-11', '2020-09-13

No data for ticker 'CNYF' in ['2020-08-17', '2020-09-05', '2020-09-10', '2020-09-11', '2020-09-06', '2020-09-08', '2020-09-09', '2020-09-12', '2020-09-13', '2020-09-14', '2020-09-15', '2020-09-17', '2020-09-16', '2020-09-19', '2020-09-20']
No data for ticker 'COGN' in ['2020-08-15', '2020-08-16', '2020-08-11', '2020-08-14', '2020-09-04', '2020-09-05', '2020-09-10', '2020-09-09', '2020-09-07', '2020-09-08', '2020-09-12', '2020-09-11', '2020-09-15', '2020-09-17', '2020-09-16', '2020-09-19', '2020-09-22', '2020-09-21']
No data for ticker 'COLY' in ['2020-09-05']
No data for ticker 'CONV' in ['2020-09-17']
No data for ticker 'COOK' in ['2020-09-05', '2020-09-18', '2020-09-20']
No data for ticker 'COPE' in ['2020-08-26', '2020-08-25', '2020-08-17', '2020-08-13', '2020-08-16', '2020-08-11', '2020-08-30', '2020-08-20', '2020-08-15', '2020-08-24', '2020-08-23', '2020-08-21', '2020-08-14', '2020-08-12', '2020-08-10', '2020-08-22', '2020-08-18', '2020-08-29', '2020-08-28', '2020-08-19', '2020-09

No data for ticker 'DMX' in ['2020-09-13']
No data for ticker 'DOCS' in ['2020-08-22', '2020-09-04']
No data for ticker 'DOSE' in ['2020-09-20']
No data for ticker 'DRAI' in ['2020-09-21', '2020-09-22']
No data for ticker 'DROV' in ['2020-09-09', '2020-09-11', '2020-09-06', '2020-09-10', '2020-09-20']
No data for ticker 'DSET' in ['2020-08-25', '2020-08-17', '2020-08-16', '2020-08-20', '2020-08-14', '2020-08-26', '2020-08-24', '2020-08-12', '2020-08-18', '2020-08-10', '2020-08-11', '2020-08-22', '2020-08-21', '2020-08-15', '2020-08-23', '2020-08-13', '2020-08-09', '2020-09-04', '2020-09-03', '2020-09-06', '2020-09-05', '2020-09-02', '2020-09-07', '2020-09-10', '2020-09-09', '2020-09-08', '2020-09-11', '2020-09-12', '2020-09-14', '2020-09-13', '2020-09-15', '2020-09-16', '2020-09-17', '2020-09-18', '2020-09-20', '2020-09-19', '2020-09-21', '2020-09-22']
No data for ticker 'DSTR' in ['2020-09-07']
No data for ticker 'DTMC' in ['2020-08-25', '2020-08-16', '2020-09-10', '2020-09-14', '2020

...

No data for ticker 'PAMM' in ['2020-08-17']
No data for ticker 'PBOC' in ['2020-08-21', '2020-08-12', '2020-08-24', '2020-08-26', '2020-08-22', '2020-08-20', '2020-08-15', '2020-08-13', '2020-08-16', '2020-08-10', '2020-08-23', '2020-08-17', '2020-08-25', '2020-08-11', '2020-08-14', '2020-09-03', '2020-09-04', '2020-09-05', '2020-09-10', '2020-09-11', '2020-09-08', '2020-09-07', '2020-09-09', '2020-09-06', '2020-09-13', '2020-09-12', '2020-09-14', '2020-09-15', '2020-09-17', '2020-09-18', '2020-09-16', '2020-09-19', '2020-09-20', '2020-09-21', '2020-09-22']
No data for ticker 'PCCG' in ['2020-08-26', '2020-08-21', '2020-08-17', '2020-09-04', '2020-09-08', '2020-09-06', '2020-09-09', '2020-09-10', '2020-09-13', '2020-09-12', '2020-09-14', '2020-09-11', '2020-09-17', '2020-09-20', '2020-09-21']
No data for ticker 'PCOR' in ['2020-09-07']
No data for ticker 'PCTL' in ['2020-08-24', '2020-08-17', '2020-08-25', '2020-08-26', '2020-08-21', '2020-08-23', '2020-08-22', '2020-09-04', '2020-09-0

No data for ticker 'RIDE' in ['2020-08-22', '2020-08-14', '2020-08-15', '2020-08-10', '2020-09-08', '2020-09-07', '2020-09-09', '2020-09-11', '2020-09-18', '2020-09-17']
No data for ticker 'RIDG' in ['2020-08-15', '2020-08-14', '2020-09-08', '2020-09-13', '2020-09-16']
No data for ticker 'RITA' in ['2020-08-13', '2020-09-11', '2020-09-15']
No data for ticker 'RIVL' in ['2020-09-15']
No data for ticker 'RNAI' in ['2020-08-25', '2020-08-26']
No data for ticker 'ROSE2' in ['2020-08-18', '2020-08-17', '2020-08-16', '2020-08-26', '2020-08-10', '2020-09-04', '2020-09-08', '2020-09-10', '2020-09-06', '2020-09-07', '2020-09-11', '2020-09-12', '2020-09-13', '2020-09-17', '2020-09-18', '2020-09-14', '2020-09-20', '2020-09-19', '2020-09-21', '2020-09-22']
No data for ticker 'ROSI' in ['2020-08-17', '2020-09-03', '2020-09-14']
No data for ticker 'ROSS' in ['2020-09-09', '2020-09-12', '2020-09-13', '2020-09-22']
No data for ticker 'RSAS' in ['2020-09-10']
No data for ticker 'RSII' in ['2020-08-21',

No data for ticker 'SLIC' in ['2020-08-22', '2020-09-13']
No data for ticker 'SLVR' in ['2020-08-17']
No data for ticker 'SMCC' in ['2020-08-16', '2020-08-11', '2020-08-17', '2020-08-26', '2020-08-25', '2020-08-12', '2020-08-23', '2020-08-15', '2020-08-20', '2020-08-10', '2020-09-05', '2020-09-08', '2020-09-10', '2020-09-06', '2020-09-11', '2020-09-09', '2020-09-07', '2020-09-13', '2020-09-14', '2020-09-12', '2020-09-17', '2020-09-15', '2020-09-16', '2020-09-18', '2020-09-20', '2020-09-19', '2020-09-21', '2020-09-22']
No data for ticker 'SMIL' in ['2020-08-17', '2020-08-25', '2020-08-26', '2020-08-16', '2020-08-20', '2020-08-23', '2020-08-21', '2020-08-13', '2020-08-24', '2020-08-15', '2020-08-22', '2020-08-10', '2020-09-04', '2020-09-03', '2020-09-06', '2020-09-05', '2020-09-08', '2020-09-09', '2020-09-10', '2020-09-07', '2020-09-11', '2020-09-13', '2020-09-12', '2020-09-14', '2020-09-16', '2020-09-17', '2020-09-18', '2020-09-15', '2020-09-19', '2020-09-20', '2020-09-21', '2020-09-22'

No data for ticker 'UGLY' in ['2020-08-24', '2020-08-23', '2020-08-25', '2020-08-11', '2020-08-17', '2020-08-12', '2020-08-13', '2020-08-14', '2020-08-16', '2020-08-26', '2020-08-15', '2020-09-05', '2020-09-04', '2020-09-03', '2020-09-08', '2020-09-06', '2020-09-09', '2020-09-10', '2020-09-07', '2020-09-11', '2020-09-12', '2020-09-14', '2020-09-13', '2020-09-15', '2020-09-17', '2020-09-18', '2020-09-16', '2020-09-19', '2020-09-20', '2020-09-21', '2020-09-22']
No data for ticker 'URT' in ['2020-09-06']
No data for ticker 'USOL' in ['2020-08-17', '2020-08-16', '2020-08-25', '2020-08-22', '2020-08-13', '2020-08-26', '2020-08-23', '2020-08-24', '2020-08-14', '2020-08-11', '2020-08-15', '2020-09-04', '2020-09-08', '2020-09-09', '2020-09-06', '2020-09-07', '2020-09-10', '2020-09-11', '2020-09-13', '2020-09-12', '2020-09-14', '2020-09-18', '2020-09-15', '2020-09-17', '2020-09-20', '2020-09-19', '2020-09-22', '2020-09-21']
No data for ticker 'USTB1' in ['2020-08-16']
No data for ticker 'VANS' 

No data for ticker 'false' in ['298.9688', '-53.5104', '631.7229', '-58.5', '-7.5656004', '956.805']
24881
Wall time: 50.9 s


In [5]:
%%time

df_stock_renamed = df_stock_data.rename(columns={"ticker": "f22_ticker"})

if 'None' in df_stock_renamed.columns:
    df_stock_renamed = df_stock_renamed.drop(columns=['None'])

df_stock_clean = df_stock_renamed.drop(columns=['lastupdated'])
print(f'Twitter: {df_twitter.shape[0]}')
print(f'Quandl: {df_stock_renamed.shape[0]}')

print(f'{df_twitter.columns}')
print(f'{df_stock_renamed.columns}')

df_merged = pd.merge(df_twitter, df_stock_renamed, how='inner', left_on=["f22_ticker","date"], right_on=["f22_ticker","date"])

df_merged_clean = df_merged.sample(frac=1.0)

print(f'Num merged: {df_merged.shape[0]}')

Twitter: 4354341
Quandl: 24881
Index(['created_at', 'created_at_timestamp', 'user_screen_name', 'favorite_count', 'in_reply_to_screen_name', 'user_verified', 'metadata_result_type', 'user_listed_count', 'user_time_zone', 'user_has_extended_profile', 'in_reply_to_status_id', 'user_statuses_count', 'user_protected', 'user_is_translation_enabled', 'user_location', 'lang', 'user_geo_enabled', 'place_country', 'place_name', 'possibly_sensitive', 'user_friends_count', 'retweet_count', 'user_follow_request_sent', 'user_followers_count', 'f22_ticker', 'f22_has_cashtag', 'f22_ticker_in_text', 'f22_num_other_tickers_in_tweet', 'f22_sentiment_pos', 'f22_sentiment_neu', 'f22_sentiment_neg', 'f22_sentiment_compound', 'date', 'f22_compound_score'], dtype='object')
Index(['f22_ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'closeunadj', 'lastupdated', 'future_open', 'future_low', 'future_high', 'future_close', 'future_date'], dtype='object')
Num merged: 1157754
Wall time: 27.

In [156]:
%%time

df_thin_rabbit = df_merged

from sklearn.preprocessing import StandardScaler

# df_thin_rabbit['stock_val_change'] = ((df_thin_rabbit['future_high'] - df_thin_rabbit['close']) / df_thin_rabbit['future_high']) * 100.0
df_thin_rabbit['stock_val_change'] = ((df_thin_rabbit['future_open'] - df_thin_rabbit['close']) / df_thin_rabbit['future_open']) * 100.0

scaler = StandardScaler()
df_thin_rabbit['stock_val_change_scaled'] = scaler.fit_transform(df_thin_rabbit[['stock_val_change']])

Wall time: 104 ms


In [157]:
%%time

df_thin_rabbit['buy_sell'] = df_thin_rabbit['stock_val_change_scaled'].apply(lambda x: 1 if x >=0 else -1)

zero_fill_cols = ['created_at_timestamp',
       'favorite_count', 
       'user_listed_count', 
       'user_statuses_count',  
       'user_friends_count',
       'retweet_count',  'user_followers_count',
       'f22_num_other_tickers_in_tweet', 'f22_sentiment_pos',
       'f22_sentiment_neu', 'f22_sentiment_neg', 'f22_sentiment_compound',
       'f22_compound_score']

# 'user_verified', 'user_time_zone', 'in_reply_to_status_id','user_has_extended_profile', 'user_protected', 
# 'user_follow_request_sent', 'f22_has_cashtag', 'f22_ticker_in_text', 'user_geo_enabled', 'possibly_sensitive'
# 'user_is_translation_enabled',

# display(df_thin_rabbit["created_at_timestamp"].head(100))

# for z in zero_fill_cols[:1]:
#     df_thin_rabbit[z][df_thin_rabbit[z].notnull()].median()

zero_fills = {z: df_thin_rabbit[z][df_thin_rabbit[z].notnull()].median() for z in zero_fill_cols}

for k, v in zero_fills.items():
    df_thin_rabbit[k] = df_thin_rabbit[k].fillna(v)

Wall time: 3.04 s


In [158]:
%%time

df_green_hornet = df_thin_rabbit.dropna(subset=["f22_ticker"])

empty_fill_cols = ['user_screen_name',
       'in_reply_to_screen_name',
       'metadata_result_type',
       'user_location', 'place_country',
       'place_name']

df_green_hornet[empty_fill_cols] = df_green_hornet[empty_fill_cols].fillna("")
df_green_hornet["lang"] = df_green_hornet["lang"].fillna("en")

print(f"df_green_hornet cols: {df_green_hornet.columns}")

df_green_hornet cols: Index(['created_at', 'created_at_timestamp', 'user_screen_name', 'favorite_count', 'in_reply_to_screen_name', 'user_verified', 'metadata_result_type', 'user_listed_count', 'user_time_zone', 'user_has_extended_profile', 'in_reply_to_status_id', 'user_statuses_count', 'user_protected', 'user_is_translation_enabled', 'user_location', 'lang', 'user_geo_enabled', 'place_country', 'place_name', 'possibly_sensitive', 'user_friends_count', 'retweet_count', 'user_follow_request_sent', 'user_followers_count', 'f22_ticker', 'f22_has_cashtag', 'f22_ticker_in_text', 'f22_num_other_tickers_in_tweet', 'f22_sentiment_pos', 'f22_sentiment_neu', 'f22_sentiment_neg', 'f22_sentiment_compound', 'date', 'f22_compound_score', 'open', 'high', 'low', 'close', 'volume', 'dividends', 'closeunadj', 'lastupdated', 'future_open', 'future_low', 'future_high', 'future_close', 'future_date', 'stock_val_change', 'stock_val_change_scaled', 'buy_sell'], dtype='object')
Wall time: 3.41 s


In [159]:
%%time

from pyspark.sql import types as T
from ams.utils import date_utils

def is_after_nasdaq_closed(created_at_timestamp: str):
    return date_utils.is_after_nasdaq_closed(utc_timestamp=created_at_timestamp)

col_after_hours = "f22_is_tweet_after_hours"
df_green_hornet[col_after_hours] = df_green_hornet.apply(lambda x: is_after_nasdaq_closed(x['created_at_timestamp']), axis=1)

df_green_hornet[[col_after_hours]].sample(frac=.0001).head()

Wall time: 1min 3s


,f22_is_tweet_after_hours
1038617,False
295218,False
886469,False
383966,False
796321,False


In [160]:
%%time

df_pay = df_green_hornet

col_purch_date = "purchase_date"
df_pay[col_purch_date] = df_pay["date"]
df_pay.loc[df_pay[col_after_hours] == True, col_purch_date] = df_pay.loc[df_pay[col_after_hours] == True, "future_date"]

display(df_pay[["date", col_purch_date, col_after_hours]].head(5))

,date,purchase_date,f22_is_tweet_after_hours
0,2020-08-26,2020-08-26,False
1,2020-08-26,2020-08-26,False
2,2020-08-26,2020-08-26,False
3,2020-08-26,2020-08-26,False
4,2020-08-26,2020-08-26,False


Wall time: 677 ms


In [161]:
%%time
# For categorizing user_screen_name

# df_g_u = df_before_close.groupby("user_screen_name").size()

# df_num_tweets = df_g_u.to_frame().rename(columns={0: 'num_tweets'}).sort_values(by="num_tweets", ascending=False)

# df_res = df_before_close.join(df_num_tweets, on=["user_screen_name"], how="inner")

# max_rabble_tweets = 200

# df_lt_rabble = df_res[df_res["num_tweets"] < max_rabble_tweets]
# df_lt_rabble["user_screen_name"] = "one_of_the_rabble"

# df_lt_tweeters = df_res[df_res["num_tweets"] >= max_rabble_tweets]

# df_all_tweeters = pd.concat([df_lt_rabble, df_lt_tweeters], axis=0)

# df_all_tweeters[df_all_tweeters["num_tweets"] < max_rabble_tweets].head()


Wall time: 0 ns


In [162]:
%%time

from typing import List
from pandas.api.types import CategoricalDtype

df_all_tweeters = df_pay

# If test does not have category, then you'll have a hard time.
def make_one_hotted(df: pd.DataFrame, cols: List[str]):
    df_one_hots = []
    for c in cols:
        uniques = df[c].unique().tolist()
        if len(uniques) > 0:
            df[c] = df[c].astype(CategoricalDtype(uniques))
            # This is one-hot-encoding.
            df_new_cols = pd.get_dummies(df[c], prefix=c)
            df_one_hots.append(df_new_cols)

    df_one_dropped = df.drop(columns=one_hot_cols)
    df_one_hots.append(df_one_dropped)

    return pd.concat(df_one_hots, axis=1)

# NOTE: Supersparse - no one_hot_cols
omit_cols = ['created_at_timestamp', 'in_reply_to_status_id', 'place_country', 'user_time_zone', 'place_name',
             'user_location', 'metadata_result_type', 'place_name, ''place_country', 
             'lang', 'in_reply_to_screen_name', 'lastupdated', 'created_at']
one_hot_cols = []


df_one_hotted = make_one_hotted(df=df_all_tweeters, cols=one_hot_cols)

narrow_cols = list(set(df_one_hotted.columns) - set(omit_cols))

print(f"narrow_cols: {narrow_cols}")

sort_by = "f22_ticker" #'created_at_timestamp'
df_one_hotted.sort_values(by=sort_by, ascending=False, inplace=True)

df_hot_narrow = df_one_hotted[narrow_cols]
df_hot_narrow.head()

narrow_cols: ['f22_sentiment_neu', 'user_friends_count', 'stock_val_change', 'open', 'date', 'future_open', 'user_has_extended_profile', 'f22_num_other_tickers_in_tweet', 'high', 'close', 'user_screen_name', 'future_date', 'future_high', 'favorite_count', 'f22_has_cashtag', 'user_is_translation_enabled', 'dividends', 'possibly_sensitive', 'user_followers_count', 'user_protected', 'f22_is_tweet_after_hours', 'future_close', 'user_geo_enabled', 'low', 'user_listed_count', 'f22_compound_score', 'f22_ticker', 'user_statuses_count', 'purchase_date', 'f22_sentiment_pos', 'user_verified', 'f22_ticker_in_text', 'f22_sentiment_neg', 'f22_sentiment_compound', 'volume', 'user_follow_request_sent', 'future_low', 'retweet_count', 'buy_sell', 'closeunadj', 'stock_val_change_scaled']
Wall time: 5.34 s


,f22_sentiment_neu,user_friends_count,stock_val_change,open,date,future_open,user_has_extended_profile,f22_num_other_tickers_in_tweet,high,close,user_screen_name,future_date,future_high,favorite_count,f22_has_cashtag,user_is_translation_enabled,dividends,possibly_sensitive,user_followers_count,user_protected,f22_is_tweet_after_hours,future_close,user_geo_enabled,low,user_listed_count,f22_compound_score,f22_ticker,user_statuses_count,purchase_date,f22_sentiment_pos,user_verified,f22_ticker_in_text,f22_sentiment_neg,f22_sentiment_compound,volume,user_follow_request_sent,future_low,retweet_count,buy_sell,closeunadj,stock_val_change_scaled
563017,0.812,785,0.355872,17.11,2020-08-14,16.86,False,1.0,17.23,16.80,Tickeron,2020-08-17,17.000,0,True,False,0.0,False,5851,False,True,16.10,False,16.60,112,-3346.1870,ZYXI,763113,2020-08-17,0.188,False,True,0.000,-0.5719,669987.0,NaN,15.81,0.0,1,16.80,0.067199
1148727,0.922,781,-0.125078,15.77,2020-09-16,15.99,False,1.0,16.40,16.01,Tickeron,2020-09-17,17.220,0,True,False,0.0,False,6306,False,False,17.22,False,15.77,125,1121.8374,ZYXI,816366,2020-09-16,0.000,False,True,0.078,0.1779,578133.0,NaN,15.70,0.0,-1,16.01,-0.119815
568639,0.909,94,0.828729,18.15,2020-08-11,18.10,False,4.0,18.49,17.95,qU8gdezzUfzpxTr,2020-08-12,18.369,0,True,False,0.0,NaN,101,False,True,18.05,False,17.61,1,20.4323,ZYXI,3960,2020-08-12,0.000,False,True,0.091,0.2023,598052.0,NaN,17.71,6.0,1,17.95,0.251065
554554,0.887,0,-1.293661,15.40,2020-08-25,15.46,False,1.0,16.10,15.66,Gambiste1,2020-08-26,15.905,0,True,False,0.0,NaN,7030,False,False,15.62,False,15.10,99,-2963.1450,ZYXI,859170,2020-08-25,0.113,False,True,0.000,-0.4215,933290.0,NaN,15.40,2.0,-1,15.66,-0.574207
554555,0.887,3352,-1.293661,15.40,2020-08-25,15.46,False,1.0,16.10,15.66,2019girijakriz,2020-08-26,15.905,0,True,False,0.0,NaN,479,False,False,15.62,True,15.10,1,-201.8985,ZYXI,32797,2020-08-25,0.113,False,True,0.000,-0.4215,933290.0,NaN,15.40,2.0,-1,15.66,-0.574207


In [163]:
%%time

from pandas import DataFrame
from typing import List

def convert_to_bool(df: DataFrame, cols: List[str]):
    for c in cols:
        df = df.astype({c: str})
        df[c] = df[c].apply(lambda x: x.lower().strip())
        df = df.replace({c: {'true': True, 'false': False, '': False, '0': False, '1': True}})
        df[c] = df[c].fillna(False).astype('bool')
    
    return df

df_cast = convert_to_bool(df_hot_narrow, ['possibly_sensitive', 'f22_ticker_in_text', 'user_verified', 'f22_has_cashtag',
                                          'user_has_extended_profile', 'user_is_translation_enabled', 'f22_ticker_in_text',
                                          'user_protected', 'user_geo_enabled'])

df_cast["original_close_price"] = df_cast["close"]
std_cols = [
       'favorite_count', 
       'user_listed_count',
       'user_statuses_count', 
       'user_friends_count',
       'retweet_count', 'user_follow_request_sent', 'user_followers_count',
       'f22_num_other_tickers_in_tweet', 'f22_sentiment_pos',
       'f22_sentiment_neu', 'f22_sentiment_neg', 'f22_sentiment_compound',
       'f22_compound_score', 'open', 'high', 'low', 'close', 'closeunadj', 'volume']

for i, c in enumerate(std_cols):
    df_cast[c] = scaler.fit_transform(df_cast[[c]])
    
df_cast.head()


C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\site-packages\sklearn\utils\extmath.py:770: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\ProgramData\Miniconda3\envs\alpha_media_signal\lib\site-packages\sklearn\utils\extmath.py:711: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


Wall time: 20.4 s


,f22_sentiment_neu,user_friends_count,stock_val_change,open,date,future_open,user_has_extended_profile,f22_num_other_tickers_in_tweet,high,close,user_screen_name,future_date,future_high,favorite_count,f22_has_cashtag,user_is_translation_enabled,dividends,possibly_sensitive,user_followers_count,user_protected,f22_is_tweet_after_hours,future_close,user_geo_enabled,low,user_listed_count,f22_compound_score,f22_ticker,user_statuses_count,purchase_date,f22_sentiment_pos,user_verified,f22_ticker_in_text,f22_sentiment_neg,f22_sentiment_compound,volume,user_follow_request_sent,future_low,retweet_count,buy_sell,closeunadj,stock_val_change_scaled,original_close_price
563017,0.360064,-0.082486,0.355872,-0.145313,2020-08-14,16.86,False,0.056343,-0.147975,-0.146542,Tickeron,2020-08-17,17.000,-0.013969,True,False,0.0,False,-0.013129,False,True,16.10,False,-0.143802,0.012947,-0.054600,ZYXI,2.001777,2020-08-17,1.245211,False,True,-1.223260,-1.780585,-0.208638,NaN,15.81,-0.123193,1,-0.146540,0.067199,16.80
1148727,1.201344,-0.082765,-0.125078,-0.148226,2020-09-16,15.99,False,0.056343,-0.149769,-0.148256,Tickeron,2020-09-17,17.220,-0.013969,True,False,0.0,False,-0.010914,False,False,17.22,False,-0.145615,0.026934,-0.006698,ZYXI,2.163639,2020-09-16,-0.639293,False,True,-0.666281,-0.162669,-0.213809,NaN,15.70,-0.123193,-1,-0.148253,-0.119815,16.01
568639,1.101920,-0.130554,0.828729,-0.143053,2020-08-11,18.10,False,2.250377,-0.145253,-0.144048,qU8gdezzUfzpxTr,2020-08-12,18.369,-0.013969,True,False,0.0,True,-0.041122,False,True,18.05,False,-0.141596,-0.106484,-0.018506,ZYXI,-0.305663,2020-08-12,-0.639293,False,True,-0.573451,-0.110019,-0.212687,NaN,17.71,-0.122846,1,-0.144046,0.251065,17.95
554554,0.933664,-0.137093,-1.293661,-0.149030,2020-08-25,15.46,False,0.056343,-0.150417,-0.149015,Gambiste1,2020-08-26,15.905,-0.013969,True,False,0.0,True,-0.007389,False,False,15.62,False,-0.147079,-0.001041,-0.050493,ZYXI,2.293742,2020-08-25,0.493414,False,True,-1.223260,-1.456052,-0.193818,NaN,15.40,-0.123077,-1,-0.149013,-0.574207,15.66
554555,0.933664,0.096080,-1.293661,-0.149030,2020-08-25,15.46,False,0.056343,-0.150417,-0.149015,2019girijakriz,2020-08-26,15.905,-0.013969,True,False,0.0,True,-0.039282,False,False,15.62,True,-0.147079,-0.106484,-0.020890,ZYXI,-0.218013,2020-08-25,0.493414,False,True,-1.223260,-1.456052,-0.193818,NaN,15.40,-0.123077,-1,-0.149013,-0.574207,15.66


In [164]:
df_tmp = df_cast #.sample(frac=.0004)

In [165]:
%%time

# filter all times from tweet counts after close time.

df_g_counting = df_tmp.groupby(['f22_ticker', 'purchase_date'])
s_sized = df_g_counting.size()

df_renamed = s_sized.to_frame().rename(columns={0: 'f22_day_tweet_count'})

df_res = df_tmp.join(df_renamed, on=["f22_ticker", "purchase_date"], how="inner")

df_res.head()

Wall time: 3.14 s


,f22_sentiment_neu,user_friends_count,stock_val_change,open,date,future_open,user_has_extended_profile,f22_num_other_tickers_in_tweet,high,close,user_screen_name,future_date,future_high,favorite_count,f22_has_cashtag,user_is_translation_enabled,dividends,possibly_sensitive,user_followers_count,user_protected,f22_is_tweet_after_hours,future_close,user_geo_enabled,low,user_listed_count,f22_compound_score,f22_ticker,user_statuses_count,purchase_date,f22_sentiment_pos,user_verified,f22_ticker_in_text,f22_sentiment_neg,f22_sentiment_compound,volume,user_follow_request_sent,future_low,retweet_count,buy_sell,closeunadj,stock_val_change_scaled,original_close_price,f22_day_tweet_count
563017,0.360064,-0.082486,0.355872,-0.145313,2020-08-14,16.86,False,0.056343,-0.147975,-0.146542,Tickeron,2020-08-17,17.000,-0.013969,True,False,0.0,False,-0.013129,False,True,16.10,False,-0.143802,0.012947,-0.054600,ZYXI,2.001777,2020-08-17,1.245211,False,True,-1.223260,-1.780585,-0.208638,NaN,15.81,-0.123193,1,-0.146540,0.067199,16.80,1
1148727,1.201344,-0.082765,-0.125078,-0.148226,2020-09-16,15.99,False,0.056343,-0.149769,-0.148256,Tickeron,2020-09-17,17.220,-0.013969,True,False,0.0,False,-0.010914,False,False,17.22,False,-0.145615,0.026934,-0.006698,ZYXI,2.163639,2020-09-16,-0.639293,False,True,-0.666281,-0.162669,-0.213809,NaN,15.70,-0.123193,-1,-0.148253,-0.119815,16.01,2
1148726,0.352416,-0.134310,-0.125078,-0.148226,2020-09-16,15.99,False,0.056343,-0.149769,-0.148256,dakotafinancial,2020-09-17,17.220,-0.013969,True,False,0.0,False,-0.033707,False,False,17.22,False,-0.145615,-0.029015,-0.013969,ZYXI,3.230632,2020-09-16,-0.639293,False,True,0.126343,0.042969,-0.213809,NaN,15.70,-0.123193,-1,-0.148253,-0.119815,16.01,2
568639,1.101920,-0.130554,0.828729,-0.143053,2020-08-11,18.10,False,2.250377,-0.145253,-0.144048,qU8gdezzUfzpxTr,2020-08-12,18.369,-0.013969,True,False,0.0,True,-0.041122,False,True,18.05,False,-0.141596,-0.106484,-0.018506,ZYXI,-0.305663,2020-08-12,-0.639293,False,True,-0.573451,-0.110019,-0.212687,NaN,17.71,-0.122846,1,-0.144046,0.251065,17.95,2
560949,1.101920,-0.095842,0.276243,-0.143161,2020-08-12,18.10,False,2.250377,-0.145515,-0.143831,PennyStockPete,2020-08-13,18.400,-0.013969,True,False,0.0,True,-0.039462,False,False,16.95,False,-0.141378,-0.106484,-0.017767,ZYXI,-0.316238,2020-08-12,-0.639293,False,True,-0.573451,-0.110019,-0.212972,NaN,16.94,-0.122846,1,-0.143829,0.036236,18.05,2


In [225]:
from datetime import timedelta
import random
from random import shuffle

def ho_split_by_ticker(df):
    min_count = 1
    max_count = 100000
    
    # NOTE: The sort will be preserved
    df.sort_values(by=['date'], inplace=True, ascending=False)
    df_holdout_raw = df[['f22_ticker', 'purchase_date']].groupby(['f22_ticker', 'purchase_date']).filter(lambda group: len(group) >= min_count and len(group) <= max_count)

    df_g_holdout = df_holdout_raw.groupby(['f22_ticker', 'purchase_date'])

    num_groups = 50

    holdout_indexes = []
    group_count = 0
    youngest_date = '2099-01-01'
    for group_name, df_group in df_g_holdout:
        group_count += 1
        if group_count > num_groups:
            break
        for row_index, row in df_group.iterrows():
            holdout_indexes.append(row_index)

    df_holdouts = df[df.index.isin(holdout_indexes)]

    embargo_tickers = df_holdouts['f22_ticker'].unique().tolist()

    df_samp = df[~df["f22_ticker"].isin(embargo_tickers)]

    print(f"ho: {df_holdouts.shape[0]}; samp: {df_samp.shape[0]}")

    return df_samp, df_holdouts

def ho_split_by_date(df, num_holdouts = 40000):
    df.sort_values(by=['purchase_date'], inplace=True)
    
    if num_holdouts > 0:
        df_holdouts = df.iloc[-num_holdouts:]
        df_samp = df.iloc[:-num_holdouts]

        f22_dates = df_holdouts["purchase_date"].unique().tolist()
        df_samp = df_samp[~df_samp["purchase_date"].isin(f22_dates)]
    else:
        df_samp = df
        df_holdouts = None
    
    return df_samp, df_holdouts

def ho_split_by_days(df, num_days_to_pull=1):
    date_max = df["purchase_date"].max()
    date_min = df["purchase_date"].min()
    
    dt_max = date_utils.parse_std_datestring(date_max)
    dt_min = date_utils.parse_std_datestring(date_min)

    total_days = (dt_max - dt_min).days
    
    pull_days = list(range(total_days + 1))
    shuffle(pull_days)
    pull_days = pull_days[:num_days_to_pull]
    
    print(pull_days)

    days_to_pull = []
    for i in pull_days:
        rand_dt = dt_min + timedelta(days=i)
        rand_date_string = date_utils.get_standard_ymd_format(rand_dt)
        days_to_pull.append(rand_date_string)
    
    df_samp = df[~df["purchase_date"].isin(days_to_pull)]
    df_holdouts = df[df["purchase_date"].isin(days_to_pull)]
                                  
    return df_samp, df_holdouts

In [228]:
# df_samp, df_holdouts = ho_split_by_days(df_res, num_days_to_pull=6) #
# df_samp, df_holdouts = ho_split_by_ticker(df_res)
df_samp, df_holdouts = ho_split_by_date(df_res)

print(f"df_samp: {df_samp.shape[0]}; df_holdouts: {df_holdouts.shape[0]}")

df_samp: 1116102; df_holdouts: 40000


In [221]:
import math
from sklearn.model_selection import train_test_split
import numpy as np
import random

def split_by_ticker():
    col_ticker = "f22_ticker"
    tickers = df_samp[col_ticker].unique().tolist()
    random.shuffle(tickers)

    num_ticks = len(tickers)
    print(num_ticks)
    
    train_tickers = tickers[:math.floor(num_ticks * .8)]
    test_tickers = tickers[math.ceil(-(num_ticks * .2)):]

    train_set = df_samp[df_samp[col_ticker].isin(train_tickers)]
    print(f"train_set: {train_set.shape[0]}")

    test_set = df_samp[df_samp[col_ticker].isin(test_tickers)]
    print(f"test_set: {test_set.shape[0]}")

    train_set = train_set.drop(columns=[col_ticker], axis=1)
    test_set = test_set.drop(columns=[col_ticker], axis=1)
    
    return train_set, test_set

def split_by_date(df):
    df.sort_values(by=['purchase_date'], inplace=True)
    
    num_rows = df.shape[0]
    
    print(f"num_rows: {num_rows}")
    
    df_train = df.iloc[:math.ceil(num_rows * .8)]
    df_test = df.iloc[math.ceil(-(num_rows * .2)):]
    
    f22_dates = df_test["purchase_date"].unique().tolist()
    df_train = df_train[~df_train["purchase_date"].isin(f22_dates)]
    
    print(f"df_train: {df_train.shape[0]}")
    print(f"df_test: {df_test.shape[0]}")
    
    return df_train, df_test

def split_by_days(df, num_days_to_pull=1):
    date_max = df["purchase_date"].max()
    date_min = df["purchase_date"].min()
    
    dt_max = date_utils.parse_std_datestring(date_max)
    dt_min = date_utils.parse_std_datestring(date_min)

    total_days = (dt_max - dt_min).days

    days_to_pull = []
    for i in range(num_days_to_pull + 1):
        rnd_days = random.randint(0,i)

        rand_dt = dt_min + timedelta(days=rnd_days)
        rand_date_string = date_utils.get_standard_ymd_format(rand_dt)
        days_to_pull.append(rand_date_string)
    
    df_train = df[~df["purchase_date"].isin(days_to_pull)]
    df_test = df[df["purchase_date"].isin(days_to_pull)]
                                  
    return df_train, df_test

In [230]:
df_shuff = df_samp.sample(frac=1.0)

# train_set, test_set = split_by_days(df_shuff, num_days_to_pull=7) # split_by_ticker()
# train_set, test_set = split_by_ticker(df_shuff)
train_set, test_set = split_by_date(df_shuff)
 
print(f"train_set: {train_set.shape[0]}; test_set: {test_set.shape[0]}")

num_rows: 1116102
df_train: 870664
df_test: 223220
train_set: 870664; test_set: 223220


In [231]:
def balance_df(df: DataFrame):
    df_samp_buy = df[df["buy_sell"] == 1]
    df_samp_sell = df[df["buy_sell"] == -1]

    num_buy = df_samp_buy.shape[0]
    num_sell = df_samp_sell.shape[0]
    if num_buy <= num_sell:
        frac = df_samp_buy.shape[0] / df_samp_sell.shape[0]
        df_samp_sell = df_samp_sell.sample(frac=frac)
    else:
        frac = df_samp_sell.shape[0] / df_samp_buy.shape[0]
        df_samp_buy = df_samp_buy.sample(frac=frac)
    
    print(f"df_samp_buy: {df_samp_buy.shape[0]}")
    print(f"df_samp_sell: {df_samp_sell.shape[0]}")

    return pd.concat([df_samp_buy, df_samp_sell]).sample(frac=1.0)

col_ticker = "f22_ticker"

train_set_bal = balance_df(train_set)
test_set_bal = balance_df(test_set)

train_cols = narrow_cols.copy()
train_cols.append('f22_day_tweet_count')
train_cols.remove('buy_sell')
train_cols.remove('date')
train_cols.remove('purchase_date')
train_cols.remove(col_ticker)
train_cols.remove("future_open")
train_cols.remove("future_low")
train_cols.remove("future_high")
train_cols.remove("future_close")
train_cols.remove("stock_val_change_scaled")
train_cols.remove("stock_val_change")
# train_cols.remove("close")
train_cols.remove("closeunadj")
# train_cols.remove("high")
# train_cols.remove("low")
train_cols.remove("user_screen_name")
train_cols.remove("future_date")
train_cols.remove("user_follow_request_sent")

X_train = np.array(train_set_bal[train_cols])
X_test = np.array(test_set_bal[train_cols])

y_train = np.array(train_set_bal['buy_sell'])
y_test = np.array(test_set_bal['buy_sell'])

print(f"{train_set_bal.shape[0]}")

# print(train_set_bal[train_set_bal['buy_sell'] == -1].shape[0])
# train_set["buy_sell"].head()
print(train_cols)


df_samp_buy: 414747
df_samp_sell: 414747
df_samp_buy: 107603
df_samp_sell: 107603
829494
['f22_sentiment_neu', 'user_friends_count', 'open', 'user_has_extended_profile', 'f22_num_other_tickers_in_tweet', 'high', 'close', 'favorite_count', 'f22_has_cashtag', 'user_is_translation_enabled', 'dividends', 'possibly_sensitive', 'user_followers_count', 'user_protected', 'f22_is_tweet_after_hours', 'user_geo_enabled', 'low', 'user_listed_count', 'f22_compound_score', 'user_statuses_count', 'f22_sentiment_pos', 'user_verified', 'f22_ticker_in_text', 'f22_sentiment_neg', 'f22_sentiment_compound', 'volume', 'retweet_count', 'f22_day_tweet_count']


In [232]:
# df_find = df_shuff[df_shuff.isin(["GH-13254"]).any(axis=1)]

# display(train_set_bal[train_cols].info())

# df_shuff[df_shuff["possibly_sensitive"] == True].shape[0]
# display(df_shuff.iloc[df_shuff["possibly_sensitive"] == ?True, "possibly_sensitive"].head(20))

In [233]:
# Cross-validation
from sklearn.tree import DecisionTreeClassifier

def dec_tree(max_depth: int):
    decisiontree= DecisionTreeClassifier(max_depth=max_depth)
    model = decisiontree.fit(X_train, y_train)
    y_test_pred = model.predict(X_test)

    from sklearn.metrics import accuracy_score

    print(accuracy_score(y_test, y_test_pred))
    
    return model
    
# dec_tree(4)
model = dec_tree(60)

0.522141575978365


In [205]:
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import accuracy_score
# import time

# start = time.time()
# clf = MLPClassifier(hidden_layer_sizes=(29, 30, 29), max_iter=800, tol=1e-19, activation='relu', solver='adam')
# clf.fit(X_train, y_train)  # Fit data
# prediction = clf.predict(X_test)  # Predict results for x_test
# accs = accuracy_score(y_test, prediction)  # Accuracy Score
# end = time.time()

# print(f"{accs}: Elapsed time: {end-start} seconds.")

# import matplotlib.pyplot as plt

# fig, ax = plt.subplots()
# ax.set_title("MLP Classifier Loss Curve")
# ax.plot(clf.loss_curve_)

# fig.canvas.draw()

In [234]:
from ams.services import stock_action_service as sas
df_stock_splits = sas.get_splits()[["ticker", "date", "value"]]

df_holdouts_clean = df_holdouts_bal.drop(columns=["date"])

df_split_aware = pd.merge(df_holdouts_clean, df_stock_splits, how='left', left_on=["f22_ticker", "purchase_date"], right_on=["ticker", "date"])
df_splitted = df_split_aware.rename(columns={"value": "split_share_multiplier"}).drop(columns=["ticker"])
df_splitted["split_share_multiplier"] = df_splitted["split_share_multiplier"].fillna(1.0) 

print(f"Columns: {df_splitted.columns}")
print(f"Count: {df_splitted.shape[0]}")
df_splitted.head()

Columns: Index(['f22_sentiment_neu', 'user_friends_count', 'stock_val_change', 'open', 'future_open', 'user_has_extended_profile', 'f22_num_other_tickers_in_tweet', 'high', 'close', 'user_screen_name', 'future_date', 'future_high', 'favorite_count', 'f22_has_cashtag', 'user_is_translation_enabled', 'dividends', 'possibly_sensitive', 'user_followers_count', 'user_protected', 'f22_is_tweet_after_hours', 'future_close', 'user_geo_enabled', 'low', 'user_listed_count', 'f22_compound_score', 'f22_ticker', 'user_statuses_count', 'purchase_date', 'f22_sentiment_pos', 'user_verified', 'f22_ticker_in_text', 'f22_sentiment_neg', 'f22_sentiment_compound', 'volume', 'user_follow_request_sent', 'future_low', 'retweet_count', 'buy_sell', 'closeunadj', 'stock_val_change_scaled', 'original_close_price', 'f22_day_tweet_count', 'date', 'split_share_multiplier'], dtype='object')
Count: 160382


,f22_sentiment_neu,user_friends_count,stock_val_change,open,future_open,user_has_extended_profile,f22_num_other_tickers_in_tweet,high,close,user_screen_name,future_date,future_high,favorite_count,f22_has_cashtag,user_is_translation_enabled,dividends,possibly_sensitive,user_followers_count,user_protected,f22_is_tweet_after_hours,future_close,user_geo_enabled,low,user_listed_count,f22_compound_score,f22_ticker,user_statuses_count,purchase_date,f22_sentiment_pos,user_verified,f22_ticker_in_text,f22_sentiment_neg,f22_sentiment_compound,volume,user_follow_request_sent,future_low,retweet_count,buy_sell,closeunadj,stock_val_change_scaled,original_close_price,f22_day_tweet_count,date,split_share_multiplier
0,0.031200,-0.104677,-0.446828,-0.157855,11.19,True,-0.675001,-0.160549,-0.158600,luWizz9,2020-08-18,11.24,-0.013969,False,False,0.0,True,-0.037767,False,True,10.915,True,-0.155576,-0.104332,-0.021605,FLEX,-0.263056,2020-08-18,1.676241,False,True,-1.223260,-1.280192,-0.173793,NaN,10.84,-0.123193,-1,-0.158598,-0.244924,11.24,6210,NaN,1.0
1,-1.070112,-0.067670,0.046115,0.004081,86.74,True,2.250377,0.002157,0.005053,TheSteveCop,2020-09-18,87.94,0.007975,False,False,0.0,True,-0.029053,False,True,86.110,True,0.005735,0.045225,-0.000478,PTC,-0.302481,2020-09-18,-0.639293,False,True,1.454525,0.876957,-0.203724,NaN,84.59,-0.123193,-1,0.005055,-0.053248,86.70,95,NaN,1.0
2,1.140160,-0.132015,-0.378215,-0.153399,13.22,True,-0.675001,-0.155926,-0.154198,xMsShannonigans,2020-09-17,13.40,-0.013969,False,False,0.0,True,-0.041511,False,True,12.920,False,-0.151338,-0.107559,-0.018697,ESSA,-0.314934,2020-09-17,-0.639293,False,True,-0.609155,-0.270343,-0.245204,NaN,12.59,-0.123193,-1,-0.154196,-0.218244,13.27,1738,NaN,1.0
3,0.627744,0.206892,0.741840,-0.168354,6.74,False,0.787688,-0.170682,-0.168468,DavidBRobinson4,2020-09-18,6.90,-0.013969,False,False,0.0,True,-0.020850,False,True,6.690,True,-0.166061,-0.105408,-0.008378,OVID,-0.211220,2020-09-18,-0.027832,False,True,-0.566310,-0.058232,-0.221616,NaN,6.55,-0.122440,1,-0.168466,0.217279,6.69,2002,NaN,1.0
4,0.260640,-0.137093,-0.133333,-0.150878,15.00,False,0.787688,-0.152728,-0.150403,Himiyn_,2020-09-18,15.00,-0.013969,False,False,0.0,True,-0.039564,False,True,14.820,False,-0.148280,-0.107559,-0.015942,MOMO,-0.116631,2020-09-18,-0.639293,False,True,0.212033,0.783956,-0.119428,NaN,14.64,-0.123019,-1,-0.150401,-0.123025,15.02,2578,NaN,1.0


In [235]:
# print(df_holdouts_bal["purchase_date"].unique().tolist())

df_holdouts_bal.groupby(["date"]).size()


date
2020-08-17    59792
2020-08-18     3878
2020-08-27        6
2020-08-28       30
2020-09-16    10705
2020-09-17    85971
dtype: int64

In [242]:
from statistics import mean
from sklearn.metrics import accuracy_score
df_holdouts_tweet_bal = df_holdouts[df_holdouts["f22_day_tweet_count"] > 0]
df_holdouts_bal = df_holdouts_tweet_bal #balance_df(df_holdouts_tweet_bal)


df_g_holdout = df_holdouts_bal.groupby(['f22_ticker', 'purchase_date'])

# model = clf

group_count = 0
acc_acc = []
group_preds = {}
for group_name, df_group in df_g_holdout:
    X_holdout = np.array(df_group[train_cols])
    y_holdout = np.array(df_group['buy_sell'])

    prediction = model.predict(X_holdout)
        
    pre_mean = sum(prediction)/len(prediction)
    pre_mean = [-1 if pre_mean < 0 else 1]
    
    pre_y_ho = sum(y_holdout)/len(y_holdout)
    pre_y_ho = [-1 if pre_y_ho < 0 else 1]

    accs_pre = accuracy_score([pre_y_ho], [pre_mean])
    acc_acc.append(accs_pre)
    
    ticker = group_name[0]
    date_str = group_name[1]
    if ticker in group_preds.keys():
        info = group_preds[ticker]
    else:
        info = {}
        group_preds[ticker] = info
    info[date_str] = pre_mean[0]
        
print(f"Mean: {mean(acc_acc)}")

g_tickers = []
for group_name, df_group in df_g_holdout:
    g_tickers.append(group_name[0])

Mean: 0.4681238615664845


In [243]:
def get_roi(close_price: float, target_roi_frac: float, future_high: float, future_close: float):
    target_price = close_price * (1 + target_roi_frac)
    
    if target_price < future_high:
        roi = target_roi_frac
    else:
        roi = (future_close - close_price) / close_price

    return roi

In [244]:
cash = 1000
num_trades_at_once = 10
roi_list = []
num_trades = 0
target_roi_frac = .06
trade_cost = 8
max_trades = 5000
min_price = 0

# NOTE: Add 'future_close' because this is the end of day price on low days that you will hit if the limit is not triggered.
# raise Exception("Need future_close. Should not be selling at high. Should be selling at target roi - close if does not reach that number.".)

# raise Exception("Should have lookup for split info to see if stock split since last activity.")
# raise Exception("For holdouts, create function to pick specified day; calculate day with biggest loss then use that day")
# raise Exception("""Remove all tweets from holdout after close time. This will prevent post close tweets from influencing purchase
#                 decision in the past. Change 'f22_day_tweet_count' to before close. """)
# raise Exception("Create holdout that contains for one stock.")
# raise Exception("Consider adding back high, close, low to be estimated.")

def get_single_attr(df: pd.DataFrame, col: str):
    col_values = df[col].tolist()
    result = None
    if len(col_values) > 0:
        result = col_values[0]
    return result



for tindex, ticker in enumerate(group_preds.keys()):
    info = group_preds[ticker]
    is_initialized = False
    num_shares = 0
    for ndx, date_str in enumerate(info.keys()):
        
        prediction = info[date_str]
        should_buy = True if prediction == 1 else False
        df_test = df_splitted[(df_splitted["f22_ticker"] == ticker) & (df_splitted["purchase_date"] == date_str)]
        
        close = get_single_attr(df_test, "original_close_price")
        if close is None:
            print("close is None.")
            continue
        
        split_share_multiplier = get_single_attr(df_test, "split_share_multiplier")
        if split_share_multiplier is None:
            print("split_share_multiplier is None.")
            continue
        split_share_multiplier = 1/split_share_multiplier
        
        future_close = get_single_attr(df_test, "future_close")
        if future_close is None:
            print("future_close is None.")
            continue
        
        future_high = get_single_attr(df_test, "future_high")
        if future_high is None:
            print("future_high is None.")
            continue
            
        future_open = get_single_attr(df_test, "future_open")
        if future_open is None:
            print("future_open is None.")
            continue
        
        num_shares = split_share_multiplier * num_shares
        
        roi = get_roi(close, target_roi_frac, future_high, future_close)
        roi_list.append(roi)
            
        if close > min_price:
            action = "bought"
            shares_price = num_trades_at_once * close
            if should_buy:
                if cash > shares_price:
                    cash -= shares_price + trade_cost
                    num_shares += num_trades_at_once
                    print(f"{ticker}; Date: {date_str}; {action}; num shares: {num_shares}; close: {close}; fo: {future_open}; fc: {future_close}; fh: {future_high}; shares_value:{num_shares * future_open}; cash:{cash}")
                    num_trades += 1
#             else:
#                 action = "sold"
#                 if num_shares > num_trades_at_once:
#                     cash += shares_price - trade_cost
#                     num_shares -= num_trades_at_once
#                     print(f"{ticker}; Date: {date_str}; {action}; num shares: {num_shares}; close: {close}; fo: {future_open}; fc: {future_close}; fh: {future_high}; shares_value:{num_shares * future_close}; cash:{cash}")
#                     num_trades += 1
            else:
                print("Signal says sell.")

            cashed_out = ((split_share_multiplier * num_shares) * future_close) - trade_cost
            total = cashed_out + cash
            cash = total
            
        if num_trades > max_trades:
            break
    
    if num_trades > max_trades:
            break
    
print(f"Total: {cash}; num_trades: {num_trades}; rois: {sum(roi_list)/len(roi_list)}")
    

Signal says sell.
Signal says sell.
AAXN; Date: 2020-09-18; bought; num shares: 10.0; close: 82.83; fo: 83.15; fc: 82.55; fh: 83.35; shares_value:831.5; cash:139.70000000000005
ABEO; Date: 2020-09-18; bought; num shares: 10.0; close: 1.96; fo: 2.2; fc: 2.08; fh: 2.335; shares_value:22.0; cash:929.6
ABIO; Date: 2020-09-18; bought; num shares: 10.0; close: 5.07; fo: 5.07; fc: 5.18; fh: 5.25; shares_value:50.7; cash:883.6999999999999
ABUS; Date: 2020-09-18; bought; num shares: 10.0; close: 2.98; fo: 2.99; fc: 3.14; fh: 3.19; shares_value:29.900000000000002; cash:881.6999999999999
Signal says sell.
ACHV; Date: 2020-09-18; bought; num shares: 10.0; close: 10.97; fo: 13.12; fc: 10.23; fh: 13.48; shares_value:131.2; cash:779.3999999999999
Signal says sell.
ACOR; Date: 2020-09-18; bought; num shares: 10.0; close: 0.58; fo: 0.599; fc: 0.54; fh: 0.61; shares_value:5.99; cash:851.8999999999999
Signal says sell.
Signal says sell.
AEIS; Date: 2020-09-18; bought; num shares: 10.0; close: 60.03; fo: 

Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Signal says sell.
Total: -3562.88; num_trades: 29; rois: 0.006226389913543395


In [ ]:
from sklearn.neural_network import MLPRegressor
from statistics import mean

def model_sales_MLP(Y, X_train, y_train, X_test, y_test, hidden, col_pred, print_coefs=True, max_iter=10000):
    import numpy as np
    model = MLPRegressor(hidden, validation_fraction=0, solver='lbfgs', max_iter=max_iter).fit(X_train, y_train)
    coefs = model.coefs_
    yhat = model.predict(X_train)
    yhatval = model.predict(X_test)
    loss = np.square(y_test - yhatval).mean()

    print('loss:', loss)
    print(f"Average off: {mean(abs(Y - yhat) / Y)}")

    return model, yhat, loss

col_pred = 'stock_val_change'
Y = train_set_bal[col_pred]
model, yhat, loss = model_sales_MLP(Y, X_train, y_train, X_test, y_test, [6], col_pred)